<a target="_blank" href="https://colab.research.google.com/github/sappw1/Dissertation/blob/main/Notebooks/Notebooks-Working/ETL/ppp_loan_preprocessing.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Basic tools
import pandas as pd
import numpy as np
import hashlib
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load labeled PPP loan data
loan_df = pd.read_csv("Data/Processed/ppp_loans_labeled29apr25.csv")
print(f" Loaded {len(loan_df)} records.")


FileNotFoundError: [Errno 2] No such file or directory: 'Data/Processed/ppp_loans_labeled29apr25.csv'

In [ ]:
# List of PII columns to hash
pii_columns = [
    "borrowername",
    "borroweraddress",
    "franchisename",
    "servicinglendername",
    "originatinglender"
]

# Hash function using SHA-256
def hash_text(val):
    if pd.isnull(val):
        return np.nan
    return hashlib.sha256(val.encode('utf-8')).hexdigest()

# Apply hashing
for col in pii_columns:
    if col in loan_df.columns:
        loan_df[col] = loan_df[col].apply(hash_text)

print("PII fields hashed successfully.")

PII fields hashed successfully.


In [ ]:
# Binary Variables → One-hot Encoding
binary_vars = ["ruralurbanindicator", "hubzoneindicator"]
loan_df = pd.get_dummies(loan_df, columns=binary_vars, drop_first=True)

# Multi-class Variables → Label Encoding (handling missing)
multi_class_vars = ["borrowerstate", "businesstype", "naicscode"]

for col in multi_class_vars:
    if col in loan_df.columns:
        # Convert everything to string to avoid mixed types
        loan_df[col] = loan_df[col].astype(str).fillna("Missing")
        loan_df[col] = loan_df[col].replace("nan", "Missing")  # Clean any str-nan
        encoder = LabelEncoder()
        loan_df[col] = encoder.fit_transform(loan_df[col])

print(" Categorical variables encoded.")


 Categorical variables encoded.


In [ ]:
# Create ForgivenessAmountRatio feature
if "forgivenessamount" in loan_df.columns and "currentapprovalamount" in loan_df.columns:
    loan_df["forgivenessamountratio"] = loan_df["forgivenessamount"] / loan_df["currentapprovalamount"]

    # Handle any weird cases like division by zero or missing values
    loan_df["forgivenessamountratio"] = loan_df["forgivenessamountratio"].replace([np.inf, -np.inf], np.nan)
    loan_df["forgivenessamountratio"] = loan_df["forgivenessamountratio"].fillna(0)

    print("ForgivenessAmountRatio feature created.")
else:
    print("ForgivenessAmountRatio could not be created — missing columns.")


ForgivenessAmountRatio feature created.


In [ ]:
# Create missingness indicators for key numeric fields
important_numeric = ["jobsreported", "currentapprovalamount"]

for col in important_numeric:
    missing_col = f"{col}_missing"
    loan_df[missing_col] = loan_df[col].isnull().astype(int)

# Drop features with excessive missingness (threshold: >30%)
missing_threshold = 0.3
missing_fractions = loan_df.isnull().mean()
columns_to_drop = missing_fractions[missing_fractions > missing_threshold].index.tolist()

loan_df = loan_df.drop(columns=columns_to_drop)

print(f" Created missingness indicators. Dropped columns: {columns_to_drop}")


 Created missingness indicators. Dropped columns: ['franchisename', 'utilities_proceed', 'mortgage_interest_proceed', 'rent_proceed', 'refinance_eidl_proceed', 'health_care_proceed', 'debt_interest_proceed', 'nonprofit']


In [ ]:
# Features to scale
numeric_features = ["currentapprovalamount", "jobsreported"]

scaler = StandardScaler()
loan_df[numeric_features] = scaler.fit_transform(loan_df[numeric_features])

print(" Numeric features scaled.")

 Numeric features scaled.


In [ ]:
# Drop normalized_borrowername
if "normalized_borrowername" in loan_df.columns:
    loan_df = loan_df.drop(columns=["normalized_borrowername"])
    print(" Dropped 'normalized_borrowername' column.")

# Save preprocessed dataset
loan_df.to_csv("Data/Processed/ppp_loans_preprocessed.csv", index=False)
print(f" Preprocessed dataset saved: {len(loan_df)} records, {loan_df['is_fraudulent'].sum()} fraud cases.")



 Dropped 'normalized_borrowername' column.
 Preprocessed dataset saved: 968525 records, 301 fraud cases.


In [ ]:
loan_df.head()

,unnamed:_0,loannumber,dateapproved,sbaofficecode,processingmethod,borrowername,borroweraddress,borrowercity,borrowerstate,borrowerzip,...,originatinglenderstate,gender,veteran,forgivenessamount,forgivenessdate,is_fraudulent,ruralurbanindicator_U,hubzoneindicator_Y,jobsreported_missing,currentapprovalamount_missing
0,0,9547507704,2020-05-01,464,PPP,ac3af1e5335672f4c44127dfcea32c215de8a22854d22e...,4cf412775ae7d7bde6a0d5a0a8ec617eb93cd3c73f44bb...,Sumter,30,29150-9662,...,GA,Unanswered,Unanswered,773553.37,11/20/2020,0,True,False,0,0
1,1,9777677704,2020-05-01,464,PPP,798a95683afc8cd1f543b4398626d3d24367fafffbe225...,2495392e77c85ab99450ef3b2135f32b7a5e3e61d84376...,North Charleston,30,29420-9000,...,GA,Male Owned,Non-Veteran,746336.24,08/12/2021,0,True,True,0,0
2,2,5791407702,2020-05-01,1013,PPP,186194fcd236b4b2c4457c3a2134addd64a5b222f5625c...,fa10ad2fa068b1eadc6e7e39acf92082a627332a928c51...,SEATTLE,30,98112-2922,...,NC,Unanswered,Unanswered,696677.49,02/10/2021,0,True,False,0,0
3,3,6223567700,2020-05-01,920,PPP,0d7ff9bd4ef379df30409ff5f929094648b6f304225c72...,0cfeef203e74ace3650feefbfaf67f01a64ecb8fe216d6...,SAN BERNARDINO,30,92407-1740,...,NC,Male Owned,Non-Veteran,395264.11,09/10/2021,0,True,False,0,0
4,4,9662437702,2020-05-01,101,PPP,c00d8bf886c74e6018aa0e135eef6daf717540d26a89f2...,NaN,NaN,30,NaN,...,OH,Unanswered,Unanswered,370819.35,04/08/2021,0,True,False,0,0


some more preprocessing prior to supervised learning

In [34]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("/content/drive/MyDrive/NCU/Dissertation/Data/Processed/ppp_loans_preprocessed.csv", low_memory=False)

# Drop any unnamed index columns if needed
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]


In [35]:
from datetime import datetime

df['dateapproved'] = pd.to_datetime(df['dateapproved'], format='%m/%d/%Y', errors='coerce')
df['loanstatusdate'] = pd.to_datetime(df['loanstatusdate'], format='%m/%d/%Y', errors='coerce')
df['forgivenessdate'] = pd.to_datetime(df['forgivenessdate'], format='%m/%d/%Y', errors='coerce')

for date_col in ['dateapproved', 'loanstatusdate', 'forgivenessdate']:
    df[f"{date_col}_year"] = df[date_col].dt.year
    df[f"{date_col}_month"] = df[date_col].dt.month
    df[f"{date_col}_day"] = df[date_col].dt.day

df.drop(columns=['dateapproved', 'loanstatusdate', 'forgivenessdate'], inplace=True)


In [36]:
HASHED_COLS = [
    'borrowername', 'borroweraddress', 'servicinglendername',
    'servicinglenderaddress', 'originatinglender'
]

def convert_hash_to_int(x):
    try:
        return int(str(x), 16) % 10**8
    except:
        return -1

for col in HASHED_COLS:
    if col in df.columns:
        df[col] = df[col].apply(convert_hash_to_int)


In [37]:
#df['borrowerzip'] = df['borrowerzip'].str[:5].astype('int', errors='ignore')
df['servicinglenderzip'] = df['servicinglenderzip'].str[:5].astype('int', errors='ignore')
df['lmiindicator'] = (df['lmiindicator'] == 'Y').astype(int)

In [38]:
df['borrowerzip'] = df['borrowerzip'].str.extract(r'(\d{5})')  # extract 5-digit ZIPs
df['borrowerzip'] = pd.to_numeric(df['borrowerzip'], errors='coerce').astype('Int64')  # nullable int type


In [39]:
cat_cols = ['processingmethod', 'borrowercity', 'borrowerstate', 'projectcity',
            'projectcountyname', 'projectstate', 'projectzip', 'businesstype',
            'gender', 'veteran', 'ethnicity', 'race', 'businessagedescription',
            'servicinglenderstate','servicinglendercity','originatinglendercity',
            'originatinglenderstate','cd','loanstatus']

for col in cat_cols:
    if col in df.columns:
        freqs = df[col].value_counts().to_dict()
        df[col + '_freq'] = df[col].map(freqs)
        df.drop(columns=[col], inplace=True)


In [40]:
numeric_cols = ['term', 'sbaguarantypercentage', 'initialapprovalamount', 'currentapprovalamount',
                'undisbursedamount', 'payroll_proceed', 'forgivenessamount', 'forgivenessamountratio']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [12]:
pd.set_option('display.max_rows', None)

# Show all columns
pd.set_option('display.max_columns', None)

# Disable column width truncation
pd.set_option('display.max_colwidth', None)

In [41]:
# Fill forgiveness-related columns with 0 (no forgiveness)
df[['forgivenessdate_year', 'forgivenessdate_month', 'forgivenessdate_day', 'forgivenessamount']] = \
    df[['forgivenessdate_year', 'forgivenessdate_month', 'forgivenessdate_day', 'forgivenessamount']].fillna(0)

# Fill loan status date columns with -1 (missing)
df[['loanstatusdate_year', 'loanstatusdate_month', 'loanstatusdate_day']] = \
    df[['loanstatusdate_year', 'loanstatusdate_month', 'loanstatusdate_day']].fillna(-1)

# Drop completely missing dateapproved splits
df.drop(columns=['dateapproved_year', 'dateapproved_month', 'dateapproved_day'], inplace=True)

# Frequency features → unseen/rare class
freq_cols = [
    'projectcountyname_freq', 'cd_freq', 'projectzip_freq',
    'projectcity_freq', 'borrowercity_freq', 'projectstate_freq',
    'businessagedescription_freq'
]
df[freq_cols] = df[freq_cols].fillna(0)

# borrowerzip: assign to dummy zip for missing (e.g., 99999)
df['borrowerzip'] = df['borrowerzip'].fillna(99999).astype(int)

# Jobs reported: unknown = -1
df['jobsreported'] = df['jobsreported'].fillna(-1)

# Handle other numeric NaNs using flag value
df['payroll_proceed'] = df['payroll_proceed'].fillna(-999)
df['undisbursedamount'] = df['undisbursedamount'].fillna(-999)

# Final check
assert not df.isnull().any().any(), "❌ Missing values remain in the DataFrame"


In [42]:


# Get count of NaNs per column
missing_summary = df.isnull().sum()
missing_summary[missing_summary > 0].sort_values(ascending=False)


,0


In [43]:
print("Before dropping is_fraudulent:")
print("df shape:", df.shape)
#print("Fraud label value counts:\n", df["is_fraudulent"].value_counts())


Before dropping is_fraudulent:
df shape: (968525, 53)


In [44]:
df.to_pickle("/content/drive/MyDrive/NCU/Dissertation/Data/Processed/ppp_loans_preprocessed_cleaned.pkl")


In [45]:
y = df["is_fraudulent"].copy()
df.drop('is_fraudulent', axis=1, inplace=True)
df.to_pickle("/content/drive/MyDrive/NCU/Dissertation/Data/Processed/X_all_scaled.pkl")
y.to_pickle("/content/drive/MyDrive/NCU/Dissertation/Data/Processed/y_labels.pkl")